In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
%aimport model

In [4]:
%aimport skorchsaver

In [5]:
import skorch

In [6]:
import numpy as np

In [7]:
import torch

In [8]:
import torchvision.datasets

In [9]:
torch.cuda.is_available()

True

In [10]:
!mkdir -p models

In [11]:
test_dataset = torchvision.datasets.FashionMNIST(
    "~/torchvision/datasets/FashionMNIST",
    download=True,
    train=False,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Pad(2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,)),
    ])
)

In [12]:
net = skorch.NeuralNetClassifier(
    module=model.Offnet(
        "fashion-mnist",
        input_size=32,
        in_channels=1,
        classes=10,
        total_block_number=24,
        widen_factor=10,
    ),
    criterion=torch.nn.CrossEntropyLoss,
    batch_size=20,
    max_epochs=10,
    optimizer=lambda params, lr: torch.optim.SGD(params, lr=lr, momentum=0.9, nesterov=True),
    callbacks=[
        ('learning_rate_decay', skorch.callbacks.LRScheduler()),
        ('progress_bar', skorch.callbacks.ProgressBar()),
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/{net.module.name}.WarmRestart.model.pt", monitor=None)),
        ('optimizer_saver', skorchsaver.OptimizerSaver(target="models/{net.module.name}.WarmRestart.optimizer.pt")),
        ('history_saver', skorchsaver.HistorySaver(target="models/{net.module.name}.WarmRestart.history.json")),
    ],
    device='cuda' if torch.cuda.is_available() else 'cpu',
    train_split=lambda train_dataset, y: (train_dataset, test_dataset),
)
net.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Offnet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): OffBlockGroup(
        (sequential): Sequential(
          (0): OffBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): FakeConv2d(
              (deform): Deform2d(
                (conv): Conv2d(16, 18, kernel_size=(1, 1), stride=(1, 1))
              )
              (conv): Conv2d(144, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
            )
            (dropout): Dropout(p=0.5)
            (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): FakeConv2d(
              (deform): Deform2d(
                (conv): Conv2d(160, 18, kernel_size=(1, 1), stride=(1, 1))
           

In [13]:
import RandomErasing.transforms

In [14]:
train_dataset = torchvision.datasets.FashionMNIST(
    "~/torchvision/datasets/FashionMNIST",
    download=True,
    train=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,)),
        RandomErasing.transforms.RandomErasing(mean=[0.4914]),
    ])
)

In [ ]:
net.fit(train_dataset, None, epochs=50)


Checkpoint! Saving model to models/Offnet-28-10-fashion-mnist-32x32x1.model.pt.
  epoch    train_loss    valid_acc    valid_loss        dur
-------  ------------  -----------  ------------  ---------
      1        0.7652       0.3004        4.0179  1600.8352



Checkpoint! Saving model to models/Offnet-28-10-fashion-mnist-32x32x1.model.pt.
      2        0.4803       0.5847        1.6770  1617.4070



Checkpoint! Saving model to models/Offnet-28-10-fashion-mnist-32x32x1.model.pt.
      3        0.4044       0.4940        2.4284  1623.9960



Checkpoint! Saving model to models/Offnet-28-10-fashion-mnist-32x32x1.model.pt.
      4        0.3565       0.7187        1.0015  1627.3563



Checkpoint! Saving model to models/Offnet-28-10-fashion-mnist-32x32x1.model.pt.
      5        0.3186       0.2729        6.1593  1635.7181


In [ ]:
# net.fit_loop(train_dataset, None, epochs=300)